# Setup Function Tools for ReAct Agent

In [ ]:
from langchain_core.tools import tool

# Tool Annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int:
    # The docstring comment describes the capabillties of the function
    # It is used by the agent to discover the function's inputs, outputs and capabillities
    """
    This function is used to add two numbers and return their sum.
    it takes two integers as inputs and return an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int:
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers and returns an integer as an output
    """
    return x * y

# Create a basic ReAct Agent

## Initiate the LLM

In [6]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


#Setup the LLM
model = ChatOpenAI(
    model="gpt-4o",   # or "gpt-4.1", "gpt-3.5-turbo", etc.
    openai_api_key=OPENAI_API_KEY, 
)


In [7]:
# Test the model
response = model.invoke("Hello, I am Akash")
print(response.content)

Hello Akash! How can I assist you today?


## Create the react agent using LangGraph

In [8]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# Create list of tools available to the agent
agent_tools = [find_sum, find_product]

# System Prompt
system_prompt = SystemMessage(
    """
    You are a mathematics expert who can solve math problems. Solve the problems provided by the user, by using only the tools available.
    Do not solve the problems yourself
    """
)

agent_graph = create_react_agent(
    model = model,
    state_modifier = system_prompt,
    tools = agent_tools

)

## Execute the agent

In [12]:
#Example 1
inputs = {"messages":[("user","what is the result of 2 * 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The result of \(2 \times 3\) is 6. 

Step by Step execution : 
================================ Human Message =================================

what is the result of 2 * 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_product (call_QqIiRY4KgGce9xR1YjMyBli2)
 Call ID: call_QqIiRY4KgGce9xR1YjMyBli2
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_product

6
================================== Ai Message ==================================

The result of \(2 \times 3\) is 6.


In [13]:
#Example 2
inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 1 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The result of \(3 \times 2\) is 6, and the result of \(5 + 1\) is also 6. 

Step by Step execution : 
================================ Human Message =================================

What is 3 multipled by 2 and 5 + 1 ?
================================== Ai Message ==================================
Tool Calls:
  find_product (call_BTXUAsVbYfb0VfUlxXcfTu4K)
 Call ID: call_BTXUAsVbYfb0VfUlxXcfTu4K
  Args:
    x: 3
    y: 2
  find_sum (call_czA2clPiL87fLHLdLJbWRCkR)
 Call ID: call_czA2clPiL87fLHLdLJbWRCkR
  Args:
    x: 5
    y: 1
================================= Tool Message =================================
Name: find_product

6
================================= Tool Message =================================
Name: find_sum

6
================================== Ai Message ==================================

The result of \(3 \times 2\) is 6, and the result of \(5 + 1\) is also 6.


# Debugging the agent

In [14]:
agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'what is the sum of 2 and 3 ?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'what is the sum of 2 and 3 ?')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='d47e7ff2-d76b-456f-88d7-c0057410e03a')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='d47e7ff2-d76b-456f-88d7-c0057410e03a')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_klDbId1KJO6DoQHEUW1RaBTG', 'function': {'arguments': '{"x":2,"y":3}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}